<a href="https://colab.research.google.com/github/murthylab/sleap-notebooks/blob/master/Training_and_inference_on_an_example_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we'll show you how to install SLEAP on Colab, download a dataset from the repository of sample datasets, run training and inference on that dataset using the SLEAP command-line interface, and then download the predictions.

# Installation

Before we install SLEAP we need to set Colab to use TensorFlow 2.

In [2]:
%tensorflow_version 2.x

Let's confirm that we have a GPU available. This next line should return something like "/device:GPU:0". If you instead see an empty string as the result, go to "Notebook settings" in the "Edit" menu and select "GPU" as the hardware accelerator.

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

Since Colab already has TensorFlow and GPU drivers installed we can install SLEAP using `pip`.

Note: This installation method should also work on other Linux machines, such as an HPC cluster, or on any system where you aren't planning to use a GPU. To use a GPU on a Windows machine you'll need to install using `conda`.

In [5]:
!pip install sleap==1.1.0a10

     |████████████████████████████████| 39.5MB 89kB/s 
     |████████████████████████████████| 61kB 10.9MB/s 
     |████████████████████████████████| 21.6MB 114kB/s 
     |████████████████████████████████| 911kB 47.9MB/s 
     |████████████████████████████████| 320.4MB 49kB/s 
     |████████████████████████████████| 829kB 50.0MB/s 
     |████████████████████████████████| 1.4MB 37.7MB/s 
     |████████████████████████████████| 235kB 60.8MB/s 
     |████████████████████████████████| 194kB 56.1MB/s 
     |████████████████████████████████| 2.3MB 51.9MB/s 
     |████████████████████████████████| 184kB 61.3MB/s 
     |████████████████████████████████| 20.1MB 1.5MB/s 
     |████████████████████████████████| 165.5MB 90kB/s 
     |████████████████████████████████| 460kB 45.6MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 849kB 55.7MB/s 
  Created wheel for pykalman: filename=pykalman-0.9.5-

## Getting your training data into Colab

You'll need to get your training data into Colab. The easiest way to do this is to export a self-contained **training package** from SLEAP and copy that onto [Google Drive](https://www.google.com/drive).

For this demo we'll instead download a sample dataset from the SLEAP [sample datasets repository](https://github.com/murthylab/sleap-datasets).

In [6]:
!apt-get install tree
!wget -O dataset.zip https://github.com/murthylab/sleap-datasets/releases/download/dm-courtship-v1/drosophila-melanogaster-courtship.zip
!mkdir dataset
!unzip dataset.zip -d dataset
!rm dataset.zip
!tree dataset

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (150 kB/s)
Selecting previously unselected package tree.
(Reading database ... 146425 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2021-02-13 17:53:50--  https://github.com/murthylab/sleap-datasets/releases/download/dm-courtship-v1/drosophila-melanogaster-courtship.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, 

# Training

Now you're ready to train a model! We'll use the command-line interface for training, and train a model for confidence maps using the default **training profile**. The training profile determines the model architecture, the learning rate for training, and other training parameters.

When you start running this cell, you'll see the training parameters listed and then you'll see the training and validation loss for each training epoch.

If you're happy with the validation loss you see for an epoch during training, you're welcome to stop training by clicking the stop button next to the notebook cell running training. The version of the model with the lowest validation loss is saved during training, and that's what will be used for inference. If you don't stop training, it will run for 200 epochs or until validation loss fails to improve for some number of epochs (controlled by the `early_stopping` fields in the training profile).

In [ ]:
!sleap-train baseline.centroid.json "dataset/drosophila-melanogaster-courtship/courtship_labels.slp" --run_name "courtship.centroid" --video-paths "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"

Let's also train a centered instance model since we'll need both a centroid model and a top-down confidence maps model to run inference with the top-down pipeline.

In [ ]:
!sleap-train baseline_medium_rf.topdown.json "dataset/drosophila-melanogaster-courtship/courtship_labels.slp" --run_name "courtship.topdown_confmaps" --video-paths "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"

The models (along with the profiles and ground truth data used to train and validate the model) are saved in the `models/` directory:

In [ ]:
!tree models/

models/
├── courtship.centroid
│   ├── best_model.h5
│   ├── initial_config.json
│   ├── labels_gt.train.slp
│   ├── labels_gt.val.slp
│   ├── training_config.json
│   └── training_log.csv
└── courtship.topdown_confmaps
    ├── best_model.h5
    ├── initial_config.json
    ├── labels_gt.train.slp
    ├── labels_gt.val.slp
    ├── training_config.json
    └── training_log.csv

2 directories, 12 files


# Inference

At this point you should have SLEAP installed, the sample dataset downloaded, and trained models for centroids and topdown confidence maps.

In [ ]:
!sleap-track "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4" --frames 0-100 -m "models/courtship.centroid" -m "models/courtship.topdown_confmaps"

When inference is finished, it will save the predictions in a file. Since we didn't specify a path, it will be saved as `<video filename>.predictions.slp` in the same directory as the video:

In [ ]:
!tree dataset/drosophila-melanogaster-courtship

dataset/drosophila-melanogaster-courtship
├── 20190128_113421.mp4
├── 20190128_113421.mp4.predictions.slp
├── courtship_labels.slp
└── example.jpg

0 directories, 4 files


You can inspect your predictions file using `sleap-inspect`:

In [ ]:
!sleap-inspect dataset/drosophila-melanogaster-courtship/20190128_113421.mp4.predictions.slp

If you're using Chrome you can download your trained models like so:

In [ ]:
# Zip up the models directory
!zip -r trained_models.zip models/

# Download.
from google.colab import files
files.download("/content/trained_models.zip")

And you can likewise download your predictions:

In [ ]:
from google.colab import files
files.download('dataset/drosophila-melanogaster-courtship/20190128_113421.mp4.predictions.slp')

In some other browsers (Safari) you might get an error and you can instead download using the "Files" tab in the side panel (it has a folder icon). Select "Show table of contents" in the "View" menu if you don't see the side panel.